In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json

from scipy.signal import find_peaks, medfilt, correlate, correlation_lags, butter, sosfilt

#import PyQt5 # для графиков в отдельных картинках
#%matplotlib qt  

%matplotlib inline
np.set_printoptions(suppress=True)

record_max_length = 1200000

picture_size_per_channel = 1.75 # резмер в высоту каждого канала на графиках (для subplots)
plt.rcParams['figure.figsize'] = [16, picture_size_per_channel]

color_spikes = 'lightgreen'
warnung_spike_lvl_multipler = 0.08 # множитель на который домнажается ампитуда сигнала для полученяи порога обнаружения выбросов 

In [2]:
class record:
    '''Вся многоканальная запись'''   
    
    name_ch_HE3 = ['ch1', 'ch2', 'ch3']
    name_ch_HE12 = ['v6', 'v5', 'v4', 'v3', 'v2', 'v1', 'II', 'I']
    
    def __init__(self, path , holter_type, amplitude_convert = 1):
               
        self.channels = []
        self.seg_with_spike = []
        croped_flag = False
        
        if holter_type == 'HE3':
            self.channel_count = 3
            time_limits_file_name = r'time_limits/time_limits_HE3.json'
            base_signal_file_name = r"base_signals/sig_base_HE3.txt"
            base_data = -np.loadtxt(base_signal_file_name, dtype=int, skiprows=0, usecols=0)
            
            for i in range(self.channel_count):
                data = np.loadtxt(path, dtype=int, skiprows=0, usecols=i)
                data = -data/amplitude_convert
                data[data>20] = 20 # ограничение по амплитуде
                data[data<-20] = -20 
                if len(data) > record_max_length:
                    data = data[0:record_max_length]
                    croped_flag = True
                self.channels.append(signal_channel(data, name = self.name_ch_HE3[i]))
            
   
        if holter_type == 'HE3BP':
            self.channel_count = 3
            time_limits_file_name = r'time_limits/time_limits_HE3BP.json'
            base_signal_file_name = r"base_signals/sig_base_HE3BP.txt"
            base_data = -np.loadtxt(base_signal_file_name, dtype=int, skiprows=0, usecols=0)             
            
            for i in range(self.channel_count):
                data = np.loadtxt(path, dtype=int, skiprows=0, usecols=i)
                data = -data/amplitude_convert
                data[data>20] = 20 # фильтрация по амплитуде
                data[data<-20] = -20 
                if len(data) > record_max_length:
                    data = data[0:record_max_length]
                    croped_flag = True
                self.channels.append(signal_channel(data, name = self.name_ch_HE3[i]))
                        
            
        if holter_type == 'HE12N':
            self.channel_count = 8
            time_limits_file_name = r'time_limits/time_limits_HE12N.json'
            base_signal_file_name = r"base_signals/sig_base_HE12N.txt"
            base_data = np.loadtxt(base_signal_file_name, dtype=int, skiprows=0, usecols=7)                 
            
            for i in reversed(range(self.channel_count)):
                data = np.loadtxt(path, dtype=int, skiprows=0, usecols=i)
                data = data/amplitude_convert
                data[data>20] = 20 # фильтрация по амплитуде
                data[data<-20] = -20 
                if len(data) > record_max_length:
                    data = data[0:record_max_length]
                    croped_flag = True
                self.channels.append(signal_channel(data, name = self.name_ch_HE12[i]))    
            
        if holter_type == 'HE12BP':
            self.channel_count = 8
            time_limits_file_name = r'time_limits/time_limits_HE12BP.json'
            base_signal_file_name = r"base_signals/sig_base_HE12BP.txt"
            base_data = np.loadtxt(base_signal_file_name, dtype=int, skiprows=0, usecols=7)     
            
            for i in reversed(range(self.channel_count)):
                data = np.loadtxt(path, dtype=int, skiprows=0, usecols=i)
                data = data/amplitude_convert
                data[data>20] = 20 # фильтрация по амплитуде\
                data[data<-20] = -20 
                if len(data) > record_max_length:
                    data = data[0:record_max_length]
                    croped_flag = True
                self.channels.append(signal_channel(data, name = self.name_ch_HE12[i])) 
           
        self.base_signal = base_data
        
        if croped_flag:
                 print('croped:', record_max_length)
        
        with open(time_limits_file_name, 'r') as f:    
             self.time_limits = (json.load(f))
      
    
    
    def find_index_lag(self, offset = 0,plot_corr = False):
        '''Получение смещения сигнала относительно базового сигнала. Смотрится первый канал у базовой записи и первый у текущей'''
        corr = correlate(self.channels[0].data, self.base_signal ,mode='full')
        lags = correlation_lags(len(self.channels[0].data), len(self.base_signal) , mode='full')
        
        self.index_lag = lags[np.argmax(np.abs(corr))] + offset
        
        if plot_corr:
            plt.rcParams['figure.figsize'] = [16, 3]
            plt.plot(lags, corr)
            plt.plot(self.index_lag, np.max(corr), 'x')
            plt.grid(True)
            plt.show()
        return self.index_lag

    def calibrate_index_lag(self, border=5000):
        '''калибровка index_lag по началу 2 участка (первого канала)'''
        calibrate_point = self.time_limits[2][1] + self.index_lag
        begin = calibrate_point - border
        end = calibrate_point + border

        temp = self.channels[0].data[begin:end]
        plt.rcParams['figure.figsize'] = [16, 3]
        plt.plot(np.arange(-border, border), temp)
        plt.axvline(x=0, color='red')
        plt.grid(True)
        plt.ylabel(f'{self.channels[0].name}, мВ',  fontsize=12)
        plt.xticks(np.arange(-border, border, len(temp)/50), rotation=45);
        plt.show()
    

    def split_channels(self):
        '''Разбивка всех каналов на сегменты по time_limits'''
        
        for i_ch in range(self.channel_count):
            self.channels[i_ch].split_signal(self.time_limits, self.index_lag)
            
    
    def find_segment_limits(self, begin, end, border = 0.5 ):
        '''постриение графика отрезка по индесам с расширенными сегментами по бокам'''
        border = int((end-begin)*border)
        print(f'begin , end: {begin}, {end}')
        print('border:', border)

        fig, axs = plt.subplots(self.channel_count, 1, figsize=(16, self.channel_count*picture_size_per_channel))
        fig.subplots_adjust(wspace=0.07, hspace=0)
        for i_ch in range(self.channel_count):
            temp = self.channels[i_ch].data[ begin - border: end + border]

            axs[i_ch].plot(np.arange(begin-border, end + border), temp)
            axs[i_ch].axvline(x=begin, color='red')
            axs[i_ch].axvline(x= end , color='red')
            axs[i_ch].grid(True)
            axs[i_ch].set_ylabel(f'{self.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
            axs[i_ch].set_xticks(np.arange(begin-border, end+ border, len(temp)/50));
            axs[i_ch].tick_params(labelrotation=45)

        temp = []
        plt.show()
        print(f'begin , end: {begin}, {end}')
        
        

        
    
    def plot(self, save_fig = False, save_fig_path = r"qqq1.png"):
        '''Постоение графика всей записи'''
        fig, axs = plt.subplots(self.channel_count, 1, figsize=(16, self.channel_count*picture_size_per_channel))
        fig.subplots_adjust(wspace=0.07, hspace=0)
        
        for i_ch in range(self.channel_count):
            temp = self.channels[i_ch].data
            axs[i_ch].plot(temp)
            axs[i_ch].grid(True)
            axs[i_ch].set_ylabel(f'{self.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
            axs[i_ch].set_xlabel('index')
        temp = []
        
        if save_fig: 
            fig.savefig(save_fig_path)
            print('saved:', save_fig_path)
            
        print('len:',self.channels[0].len)
        plt.show()
        
        
    def plot_segment(self, n_seg, border = 0.1 ):
        '''постриение графика отрезка расширенными сигментами по бокам'''
        print('Segment:' ,self.time_limits[n_seg][0])
        begin, end = self.channels[0].data_split[n_seg].index_position
        border = int((end-begin)*border)
        
        fig, axs = plt.subplots(self.channel_count, 1, figsize=(16, self.channel_count*picture_size_per_channel))
        fig.subplots_adjust(wspace=0.07, hspace=0)
        for i_ch in range(self.channel_count):
            temp = self.channels[i_ch].data[ begin - border: end + border]

            axs[i_ch].plot(np.arange(begin-border, end + border), temp)
            axs[i_ch].axvline(x=begin, color='red')
            axs[i_ch].axvline(x= end , color='red')
            axs[i_ch].grid(True)
            axs[i_ch].set_ylabel(f'{self.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
            axs[i_ch].set_xticks(np.arange(begin-border, end+ border, len(temp)/50));
            axs[i_ch].tick_params(labelrotation=45)

        temp = []
        plt.show()

In [3]:
class signal:
    '''Базовый класс для сигналов'''  
    
    list_of_2 = [2**5, 2**6, 2**7, 2**8, 2**9, 2**10, 2**11, 2**12, 2**13, 2**14, 2**15, 2**16, 2**17, 2**18, 2**19, 2**20]
   
    def __init__(self, sig):
        self.data = sig
        self.fd = 409
        self.len =  self.data.size  

    def plot_signal(self):
        '''Построение графика сигнала'''
        plt.rcParams['figure.figsize'] = [16, picture_size_per_channel]
        plt.plot(self.data)
        #plt.title('Сигнал')
        #plt.xlabel('Индекс')
        #plt.ylabel('Амплитуда, мВ')
        plt.xticks(np.arange(0, self.len, self.len/30), rotation=45);
        plt.grid(True)
        plt.show()
        
    def plot_signal_index(self, begin, end):
        '''Построение графика сигнала по индексу'''
        print('begin/end:', begin, end)
        temp = self.data[begin:end]
        plt.rcParams['figure.figsize'] = [16, 3]
        plt.plot(np.arange(begin, end) ,temp)
        plt.xticks(np.arange(begin, end, len(temp)/30), rotation=45);
        plt.grid(True)
        plt.show()
           

In [4]:
class signal_channel(signal):
    '''Сигнал с одного канала'''  
    
    def __init__(self, sig, name=''):
        self.name = name
        self.data = sig
        self.fd = 409
        self.len =  self.data.size
    
    def split_signal(self, time_limits, offset=0):
        '''Разбивка канала на отдельные сегменты с испытаниями'''
        data_split = []
        for i in range(len(time_limits)):
            data_split.append(signal_segment( self.data[ time_limits[i][1]+offset : time_limits[i][2]+offset],  [time_limits[i][1]+offset, time_limits[i][2]+offset]   ))
        self.data_split = data_split
  

In [5]:
class signal_segment(signal):
    '''Отрезки основной записи сигнала, полученные с помощью time_limits'''    
    
    def __init__(self, sig, index_position = [0, 0]):
        
        if index_position == [0,0]:
            self.index_position = [0, len(sig)]
        else:
            self.index_position = index_position
        self.data = sig
        self.fd = 409
        self.len =  self.data.size    
    
    
    def calculate_full_spectrum(self, peaks_height = 0.5):
        '''Высчитывает спектр сигнала и находит его пики не ниже чем peaks_height'''
        len_2 = min(self.list_of_2, key=lambda x:  (len(self.data)-x) if (len(self.data)-x)>=0 else np.nan) # максимальное значение 2^n меньше или равное заданному
        
        spectrum = np.abs(np.fft.rfft(self.data, n = len_2, norm='forward',))*2
        spectrum[0] = spectrum[0]/2 # нормализация нулевой частоты
        spectrum = spectrum * 2 # перевод амплитуды в размах        
        frequency_array = np.fft.rfftfreq(len_2, d = 1./self.fd)
        self.spectrum = spectrum
        self.frequency_array = frequency_array
        
        self.peaks_index, _ = find_peaks(self.spectrum, height=peaks_height)
        #print('Найденные амплитуда и частота пика:',np.around(self.spectrum[self.peaks_index],3), np.around(self.frequency_array[self.peaks_index],3))
        #return spectrum, frequency_array

    def calculate_window_spectrum(self, n = 2**11, step = 256, peaks_height = 0.5):
        '''Высчитывает массив с максимальными компонентами спектра для скользящего окна размера n с шагом step,
        Формируем массив оконного спектра self.window_spectrum_array - первый столбец - амплитуда, второй - частота, третий - смещение окна относительно начала записи '''
        
        self.window_size = n
        self.step = step
        begin = 0
        end = begin + n
        result_array = [] 
        data = self.data
        
        for offset in np.arange(0, len(data)-n, step):
    
            sig_window = signal_segment(data[begin + offset: end + offset])
            sig_window.calculate_full_spectrum(peaks_height = peaks_height);
            
            # Фильтрация по количеству найденных компонент (от 1 до 2 включительно), для отбрасывания переходных участков
            if (sig_window.peaks_index.size>0) and (sig_window.peaks_index.size<=2):
                result_array.append([
                    np.max(sig_window.spectrum[sig_window.peaks_index]),
                    sig_window.frequency_array[sig_window.peaks_index[(sig_window.spectrum[sig_window.peaks_index]).argmax()]],
                    offset,   
                    ])                            
            else:
                result_array.append([0,0, offset]) 
        self.window_spectrum_array = np.array(result_array)
        
        #return self.window_spectrum_array       
    
    
    def get_amplitude(self, n_size=5):
        '''Возвращает медианное значение по последним n_size элементам оконного спектра'''
        if self.window_spectrum_array.size !=0:
            rez = np.median(np.array(self.window_spectrum_array[:,0][-n_size:])) 
        else: 
            rez = 0
        return rez
    
    
    def check_spike(self, midifilt_size = 7, plot_spike = False, peaks_height = 0.5 , warnung_lvl=0, window_size=2**11):
        '''Проверяет на наличие выбросов (шума?), сравнивая сигнал с его отфильтрованной медианным фильтром версией, возвращает флаг наличия выбросов'''
        
        if warnung_lvl == 0:
            self.calculate_window_spectrum(peaks_height=peaks_height, n = window_size)
            warnung_lvl = self.get_amplitude()*warnung_spike_lvl_multipler
            #print('found amp:', warnung_lvl/warnung_spike_lvl_multipler)
            if warnung_lvl == 0: 
                warnung_lvl = 2*warnung_spike_lvl_multipler

        data = self.data
        data_filtred = medfilt(data, kernel_size=midifilt_size)

        data_diff = abs(data - data_filtred)
        data_diff =data_diff[5:-5]# обрезка краев для минимизации краевых эффектов
        max_diff = max(data_diff)
        if plot_spike:
            plt.rcParams['figure.figsize'] = [16, 1.5]
            plt.plot(data_diff);
            plt.title(f'warnung_lvl: {round(warnung_lvl,2)}, max diff: {round(max_diff,2)}')
            plt.ylim([ 0,  warnung_lvl*2])
            plt.grid(True)
            plt.axhline(y=warnung_lvl, color='r')
            plt.show()
        return max_diff > warnung_lvl

    
    def plot_spectrum_peaks(self):
        '''Построения графика спектра с пиками, если они были найдены'''
        plt.plot(self.frequency_array, self.spectrum)
        #plt.xlabel('Частота, Гц')
        plt.ylabel('Размах, мВ')
        plt.grid(True)
        plt.plot(self.frequency_array[self.peaks_index], self.spectrum[self.peaks_index],'x')
        for index in self.peaks_index:
            plt.text(self.frequency_array[index], self.spectrum[index], str(round(self.spectrum[index],2))+'mV, ' + str(round(self.frequency_array[index],3))+ ' Hz', size=14)
        plt.show()        
    

    def plot_window_spectrum(self):
        '''Построение графиков амплитуды и частоты главной компоненты в окне'''
        
        #window_info_str = ' (window: ' + str(round(self.window_size/409,2)) + ' sec, ' + 'step: ' +  str(round(self.step/409,2)) + ' sec)'
        if self.window_spectrum_array.size !=0:
    
            grap = self.window_spectrum_array[:,0]
            freq = self.window_spectrum_array[:,1]
            plt.plot(self.window_spectrum_array[:,2]/409 , grap)
            plt.plot(self.window_spectrum_array[:,2]/409 , grap, 'x')
            #plt.title('Измеренный размах в окне' + window_info_str, loc='left', size=12)
            #plt.xlabel('Начало окна, с', size=12)
            plt.ylabel('Размах, мВ')
            plt.grid(True)

            if len(grap)//5 == 0:
                step = 1
            else:
                step = len(grap)//5
            
            for index in np.arange(0, len(grap), step):
                plt.text(index*self.step/409, grap[index],  str(round(grap[index],2))+'mV, ' + str(round(freq[index],2))+'Hz' , size=12, rotation=15)  

        #plt.tight_layout()
        plt.show() 
        

## Функции

In [6]:
def get_index_from_time(time_start, time_end, offset):
    '''Получение индекса границы отрезка по разности времени'''
    from datetime import datetime as dt 
    
    offset = (offset/1000)
    time_start_obj = dt.strptime(time_start, '%H:%M:%S')
    time_end_obj = dt.strptime(time_end, '%H:%M:%S')
    #print('index:' ,round(((time_end_obj - time_start_obj).total_seconds()+offset)*409))
    return round(((time_end_obj - time_start_obj).total_seconds()+offset)*409)

In [7]:
def nose_test(record, max_nose = 0.02, filtred = False):
    '''Испытание на шум
    считается максимальных размах шума в двадцати отрезках по 10 сек для каждого канала
    
    Если включена фильтрация изолинии (ФВЧ баттерворда 1 порядка по 0.1 Гц) то начало участка обрезается '''
    
    fig, axs = plt.subplots(record.channel_count, 1, figsize=(16, record.channel_count*picture_size_per_channel))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    i_seg = 0
    data_max_nose = 0
    rez_array = []
    for i_ch in range(record.channel_count):
        temp = record.channels[i_ch].data_split[i_seg]
        data = temp.data
        
        if filtred:
            sos = butter(1, 0.1, 'hp', fs=409, output='sos')
            data = sosfilt(sos, data)
            
            #sos = butter(1, 160, 'lp', fs=409, output='sos')
            #data = sosfilt(sos, data)
        
        axs[i_ch].plot(np.arange(temp.data.size)/409, data)
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{record.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
        axs[i_ch].set_xlabel('sec')    
   
        if filtred: 
            # убираем 15 секуд начала записи
            begin_offset = 15*409
            offset_range = range( begin_offset, (temp.len - 10*409),  round((temp.len-begin_offset)/20))
        else:
            offset_range = range( 0, (temp.len - 10*409),  round(temp.len/20))

            
        for offset in offset_range: 
            temp_window = data[0+offset: 10*409 + offset]

            temp_window_diff = (temp_window.max() - temp_window.min())

            if temp_window_diff > data_max_nose:
                data_max_nose = temp_window_diff

            temp_window_diff = 0
            temp_window = []
        rez_array.append(round(data_max_nose,3))
        data_max_nose = 0
    temp = []
    print('Максимальный размах записи:   ', round(max(rez_array),3), 'мВ')
    print('Максимально допустимый размах:', max_nose, 'мВ')
    print(rez_array)
    return  rez_array

In [8]:
def amplitude_test(record, n_seg,  n_size = 5, peaks_height = 0.5, window_size = 2**11, facecolor = 'white', check_spike=True):
    '''Амплитудный тест, строит графики отрезка сигнала по всем каналам с добавлением значений амплитуд и частот,
    возвращает массив с медианным значением последних n_size значений оконного спектра для каждого канала, 
    также возвращает наличие выбросов'''
    
    fig, axs = plt.subplots(record.channel_count, 1, figsize=(16, record.channel_count*picture_size_per_channel))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    fig.suptitle('Segment: ' + record.time_limits[n_seg][0] , fontsize=14)
    fig.patch.set_facecolor(facecolor)
    rez_array = []
    median_array = []
    check_spikes_warnung_flag = False
    
    for i_ch in range(record.channel_count):
        temp = record.channels[i_ch].data_split[n_seg]

        axs[i_ch].plot(np.arange(temp.data.size)/409, temp.data)
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{record.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
        
        if check_spike:
            '''проверка на выбросы/искажения'''
            if temp.check_spike():
                axs[i_ch].set_facecolor(color_spikes)
                check_spikes_warnung_flag = True
                record.seg_with_spike.append([n_seg, i_ch])

        temp.calculate_window_spectrum(peaks_height = peaks_height, n = window_size)

        if temp.window_spectrum_array.size !=0:

            grap = temp.window_spectrum_array[:,0]
            freq = temp.window_spectrum_array[:,1]
            
            rez_array.append(np.array(grap[-n_size:]))                   
   
            if len(grap)//6 == 0:
                step = 1
            else:
                step = len(grap)//6
            
            axs[i_ch].set_ylim(top = axs[i_ch].get_ylim()[1]*1.3)
            for index in np.arange(0 ,len(grap), step):
                text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                limit_diff = axs[i_ch].get_ylim()[1] - axs[i_ch].get_ylim()[0]
                text_y = axs[i_ch].get_ylim()[1] - limit_diff * 0.4                
                axs[i_ch].text(index*temp.step/409, text_y,  text_str , size=12, rotation=15)  
        else: 
            for i in range(record.channel_count):
                rez_array.append(0)
             
    axs[2].set_xlabel(f'sec, (размер окна: {window_size}, {round(window_size/409,2)} sec) ')
    
    temp = []

    #fig.tight_layout()   
    
    #fig.savefig(r"qqq1.png",)
    
    for i in range(record.channel_count):
        median_array.append(round(np.median(rez_array[i]),2))
    plt.show()
    print(median_array)
    return median_array, check_spikes_warnung_flag

In [9]:
def ECG_test(record, n_seg):
    '''Тест сегмента с ЭКГ, считает точность по времени и выводит график отрезка с переключением полярности каналов,
    возвращает результат пргохождения теста на точность времени'''
    warnung_flag = 0
    
    fig, axs = plt.subplots(record.channel_count, 1, figsize=(16, record.channel_count*picture_size_per_channel))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    for i_ch in range(record.channel_count):
        temp = record.channels[i_ch].data_split[n_seg].data
        peaks_index, _ = find_peaks(temp, distance = 0.95*409, )
        axs[i_ch].plot(np.arange(temp.size)/409, temp)
        axs[i_ch].plot(peaks_index/409, temp[peaks_index], 'x')
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{record.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
        axs[i_ch].set_xlabel('sec')

        print(f'{record.channels[i_ch].name}, max = {round(max(np.diff(peaks_index))/409,3)} sec, min = {round(min(np.diff(peaks_index))/409,3)} sec')
        if abs((max(np.diff(peaks_index))/409)-1.0 > 0.05)  or  abs((min(np.diff(peaks_index))/409)-1.0 > 0.05):
            warnung_flag = 1
    temp = []        

    print('Прохождение испытания:', warnung_flag==0)

    n_seg = n_seg + 1
    fig, axs = plt.subplots(record.channel_count, 1, figsize=(16, record.channel_count*picture_size_per_channel))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    for i_ch in range(record.channel_count):
        temp = record.channels[i_ch].data_split[n_seg].data
        axs[i_ch].plot(np.arange(temp.size)/409, temp)
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{record.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
        axs[i_ch].set_xlabel('sec')
    temp = []
    return warnung_flag == 0

In [10]:
def doptok_test(record, n_seg):
    '''Испытание на дополнительный ток в цепи пациента, считает максимальный размах сигнала на канале и сравнивает с 1 мВ'''
    
    fig, axs = plt.subplots(record.channel_count, 1, figsize=(16, record.channel_count*picture_size_per_channel))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    print('Segment:' ,record.time_limits[n_seg][0])

    ymax_for_plot = 0
    ymin_for_plot = 0
    warnung_flag = 0
    rezult_array = []

    for i_ch in range(record.channel_count):
        temp = record.channels[i_ch].data_split[n_seg].data
        temp = temp - np.median(temp)
        temp_max = max(temp)
        temp_min = min(temp)
        
        rezult_array.append([round(temp_max,2), round(temp_min,2)])
        
        if ymax_for_plot < temp_max * 1.5:    ymax_for_plot = temp_max * 1.5   
        if ymin_for_plot > temp_min * 1.5:    ymin_for_plot = temp_min * 1.5

        if (abs(temp_max) > 1.0) or (abs(temp_min) > 1):
            warnung_flag = 1

        print((f'{record.channels[i_ch].name}, max = {round(temp_max,2)} мВ, min = {round(temp_min,2)} мВ'))

    print('Прохождение испытания:', warnung_flag==0)

    for i_ch in range(record.channel_count):
        temp = record.channels[i_ch].data_split[n_seg].data
        #temp = temp - np.median(temp) # Корректировка уровня (удаление постоянной составляющей)
        axs[i_ch].plot(np.arange(temp.size)/409, temp)
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{record.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
        axs[i_ch].set_xlabel('sec')
        axs[i_ch].set_ylim([ymin_for_plot + np.median(temp), ymax_for_plot + np.median(temp)])

    temp = []
    return warnung_flag == 0, rezult_array

In [11]:
def impedanse_test_a(record, n_size = 5 , n_seg = 11, imp_ch = 0, peaks_height = 0.5, window_size = 2**11, facecolor = 'white', ):
    '''Часть теста входных сопротивлений самостоятельно не применяется, но используется в функции impedanse_test(),
    строит график для одного электрода, imp_ch задает порядок электрода для смены цвета'''
    
    
    #rez_array = np.empty((0, n_size), int)
    rez_array = []
    fig, axs = plt.subplots(3, 2, figsize=(16, 6))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    #fig.suptitle('Segment: ' + record.time_limits[n_seg][0] , fontsize=14)
    fig.patch.set_facecolor(facecolor)
    check_spikes_warnung_flag = False
    
    
    for offset_seg in range(2):
        n_seg = n_seg + offset_seg

        axs[0, offset_seg].set_title('Segment: ' + record.time_limits[n_seg][0] , fontsize=12)
        for i_ch in range(3):
            temp = record.channels[i_ch].data_split[n_seg]
            plot_color = None
            if imp_ch == i_ch: plot_color = 'tomato'

            axs[i_ch, offset_seg].plot(np.arange(temp.data.size)/409, temp.data, color=plot_color)
            axs[i_ch, offset_seg].grid(True)
            
            if temp.check_spike():
                axs[i_ch, offset_seg].set_facecolor(color_spikes)
                check_spikes_warnung_flag = True
                record.seg_with_spike.append([n_seg, i_ch])
 
            temp.calculate_window_spectrum(peaks_height = peaks_height, n = window_size)

            if temp.window_spectrum_array.size !=0:

                grap = temp.window_spectrum_array[:,0]
                freq = temp.window_spectrum_array[:,1]
                   
                rez_array.append(np.array(grap[-n_size:]))

                if len(grap)//5 == 0:
                    step = 1
                else:
                    step = len(grap)//5

                axs[i_ch, offset_seg].set_ylim(top = axs[i_ch, offset_seg].get_ylim()[1]*1.3)
                for index in np.arange(0 ,len(grap), step):
                    text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                    limit_diff = axs[i_ch, offset_seg].get_ylim()[1] - axs[i_ch, offset_seg].get_ylim()[0]
                    text_y = axs[i_ch, offset_seg].get_ylim()[1] - limit_diff * 0.3                
                    axs[i_ch, offset_seg].text(index*temp.step/409 , text_y,  text_str , size=10, rotation=15)        

        axs[2, offset_seg].set_xlabel(f'sec, (размер окна: {window_size}, {round(window_size/409,2)} sec) ')
        temp = []

    #fig.tight_layout()   
    #fig.savefig(r"qqq1.png",)
    return rez_array, check_spikes_warnung_flag

In [12]:
def impedanse_test(record_3ch, n_size = 5, start_n_seg = 11): 
    '''Основной тест на входные сопротивления'''

    list_max = []
    list_min = []
    check_spikes_warnung_flag = False
    
    for i in range(3):
        impedanse_array, temp_spikes_warnung_flag = impedanse_test_a(record_3ch, n_size = n_size, n_seg = start_n_seg + i*2, imp_ch = i, facecolor = 'white')
        
        if temp_spikes_warnung_flag:
            check_spikes_warnung_flag = True
        
        temp_list_max = []
        for j in range(len(impedanse_array)):
            temp_list_max.append(np.max(impedanse_array[j]))

        list_max.append(temp_list_max)
        list_min.append([impedanse_array[i].min(), impedanse_array[i+3].min()])  

    base_impedanse = []
    base_impedanse.append(list_max[1][3])
    base_impedanse.append(list_max[0][4])
    base_impedanse.append(list_max[1][5])

    rezult_impedanse = []

    for i in range(3):
        rezult_impedanse.append([round(0.62*list_min[i][0]/(base_impedanse[i]-list_min[i][0])),  round(0.62*list_min[i][1]/(base_impedanse[i]-list_min[i][1]))])
    print(rezult_impedanse)
    return rezult_impedanse, check_spikes_warnung_flag

In [13]:
def impedanse_12_std(record, n_seg, n_size = 5, peaks_height = 0.5, window_size = 2**11, plot_base = False):
    '''Функция расчитывает n_size последних значений оконного спектра для I и II каналов на 3 последоватльных участках начиная с n_seg (R, L, F),
    и высчитывает входное сопротивление, сравнивая с участками без доп. сопротивления'''
    rez_array = []
    check_spikes_warnung_flag = False
    
    for offset_seg in range(3):
        i_seg = n_seg + offset_seg

        fig, axs = plt.subplots(2, 1, figsize=(16, 3))
        fig.subplots_adjust(wspace=0.07, hspace=0)

        axs[0].set_title('Segment: ' + record.time_limits[i_seg][0] , fontsize=12)

        for i_ch in range(2):
            temp = record.channels[i_ch].data_split[i_seg]

            plot_color = None
            axs[i_ch].plot(np.arange(temp.data.size)/409, temp.data, color=plot_color)
            axs[i_ch].grid(True) 
            axs[i_ch].set_ylabel(f'{record.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
            
            if temp.check_spike():
                axs[i_ch].set_facecolor(color_spikes)
                check_spikes_warnung_flag = True
                record.seg_with_spike.append([i_seg, i_ch])
            
            temp.calculate_window_spectrum(peaks_height = peaks_height, n = window_size)

            if temp.window_spectrum_array.size !=0:
                grap = temp.window_spectrum_array[:,0]
                freq = temp.window_spectrum_array[:,1]
                rez_array.append(np.median(np.array(grap[-n_size:])))

                if len(grap)//7 == 0:
                    step = 1
                else:
                    step = len(grap)//7

                axs[i_ch].set_ylim(top = axs[i_ch].get_ylim()[1]*1.3)
                for index in np.arange(0 ,len(grap), step):
                    text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                    limit_diff = axs[i_ch].get_ylim()[1] - axs[i_ch].get_ylim()[0]
                    text_y = axs[i_ch].get_ylim()[1] - limit_diff * 0.4                
                    axs[i_ch].text(index*temp.step/409 , text_y,  text_str , size=10, rotation=15)        

        axs[1].set_xlabel(f'sec, (размер окна: {window_size}, {round(window_size/409,2)} sec) ')
        temp = []
    
    # получение базовых амплитуд
    rez_array_base = []
    if n_seg == 17: i_seg=44
    if n_seg == 26: i_seg=45
    if n_seg == 35: i_seg=46
    #print("base", i_seg )
    
    if plot_base:    
        fig, axs = plt.subplots(2, 1, figsize=(16, 3))
        fig.subplots_adjust(wspace=0.07, hspace=0)
        axs[0].set_title('Segment: ' + record.time_limits[i_seg][0] , fontsize=12)
        
    for i_ch in range(2):
        temp = record.channels[i_ch].data_split[i_seg]
        temp.calculate_window_spectrum(peaks_height = peaks_height, n = window_size)
        
        if plot_base:   
            axs[i_ch].plot(np.arange(temp.data.size)/409, temp.data, color=plot_color)
            axs[i_ch].grid(True) 
            
            if temp.check_spike():
                axs[i_ch].set_facecolor(color_spikes)
                check_spikes_warnung_flag = True
                
            axs[i_ch].set_ylabel(f'{record.channels[i_ch].name}, мВ', rotation=0, fontsize=12)
        
        
        if temp.window_spectrum_array.size !=0:
            grap = temp.window_spectrum_array[:,0]
            rez_array_base.append(np.median(np.array(grap[-n_size:])))
            
            if plot_base:
                freq = temp.window_spectrum_array[:,1]
                if len(grap)//7 == 0:
                    step = 1
                else:
                    step = len(grap)//7

                axs[i_ch].set_ylim(top = axs[i_ch].get_ylim()[1]*1.3)
                for index in np.arange(0 ,len(grap), step):
                    text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                    limit_diff = axs[i_ch].get_ylim()[1] - axs[i_ch].get_ylim()[0]
                    text_y = axs[i_ch].get_ylim()[1] - limit_diff * 0.4                
                    axs[i_ch].text(index*temp.step/409 , text_y,  text_str , size=10, rotation=15) 
                
        axs[1].set_xlabel(f'sec, (размер окна: {window_size}, {round(window_size/409,2)} sec) ')    
    
    
    # расчет импеданса 
    impedanse_array = []
    voltage_array = []
    # для R контакта выбор между 2 значениями
    if rez_array[0] < rez_array[1]:
        i_ch = 0
        i_ch_base = 0
        voltage_array.append([round(rez_array[i_ch],3), round(rez_array_base[i_ch_base],3)])
        impedanse_array.append(round(0.62*rez_array[i_ch]/(rez_array_base[i_ch_base]-rez_array[i_ch])))    
    else:
        i_ch = 1
        i_ch_base = 1
        voltage_array.append([round(rez_array[i_ch],3), round(rez_array_base[i_ch_base],3)])
        impedanse_array.append(round(0.62*rez_array[i_ch]/(rez_array_base[i_ch_base]-rez_array[i_ch]))) 
    
    # для L контакта выбор между 2 значениями
    i_ch = 2
    i_ch_base = 1
    voltage_array.append([round(rez_array[i_ch],3), round(rez_array_base[i_ch_base],3)])
    impedanse_array.append(round(0.62*rez_array[i_ch]/(rez_array_base[i_ch_base]-rez_array[i_ch])))     
    
    # для F контакта выбор между 2 значениями
    i_ch = 5
    i_ch_base = 0
    voltage_array.append([round(rez_array[i_ch],3), round(rez_array_base[i_ch_base],3)])
    impedanse_array.append(round(0.62*rez_array[i_ch]/(rez_array_base[i_ch_base]-rez_array[i_ch])))
    
    
    print('Значения напряжения:',voltage_array)       
    print('Полученные входные сопротивления:',impedanse_array)
    plt.show()
    return impedanse_array, check_spikes_warnung_flag

In [14]:
def impedanse_12_v1_v6(record, n_seg, n_size = 5, peaks_height = 0.5, window_size = 2**11, plot_base = False):
    '''Функция расчитывает n_size последних значений оконного спектра для V1 - V6. График и значения на выходе 
    собраны из 6 сегментов, для каждого канала выбран сегмент, где на на нем стоит дополнительное сопротивление,
    крч каждый график это график с доп. сопротивлением'''
    check_spikes_warnung_flag = False
    rez_array = []

    fig, axs = plt.subplots(6, 1, figsize=(16, 6*1.5))
    fig.subplots_adjust(wspace=0.07, hspace=0)

    axs[0].set_title(f'Seg:    {record.time_limits[n_seg][0]}    -    {record.time_limits[n_seg+5][0]}  (сборный, на каждый канал новый сегмент)', fontsize=12)

    for ch_offset in range(6):
        temp = record.channels[ch_offset+2].data_split[n_seg + ch_offset]

        plot_color = None
        axs[ch_offset].plot(np.arange(temp.data.size)/409, temp.data, color=plot_color)
        axs[ch_offset].grid(True) 
        axs[ch_offset].set_ylabel(f'{record.channels[ch_offset+2].name}, мВ', rotation=0, fontsize=12)
        
        if temp.check_spike():
            axs[ch_offset].set_facecolor(color_spikes)
            check_spikes_warnung_flag = True
            record.seg_with_spike.append([n_seg + ch_offset, ch_offset+2])
            
        temp.calculate_window_spectrum(peaks_height = peaks_height, n = window_size)

        if temp.window_spectrum_array.size !=0:

            grap = temp.window_spectrum_array[:,0]
            freq = temp.window_spectrum_array[:,1]
            rez_array.append(np.median(np.array(grap[-n_size:])))

            if len(grap)//7 == 0:
                step = 1
            else:
                step = len(grap)//7

            axs[ch_offset].set_ylim(top = axs[ch_offset].get_ylim()[1]*1.3)
            for index in np.arange(0 ,len(grap), step):
                text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                limit_diff = axs[ch_offset].get_ylim()[1] - axs[ch_offset].get_ylim()[0]
                text_y = axs[ch_offset].get_ylim()[1] - limit_diff * 0.4                
                axs[ch_offset].text(index*temp.step/409 , text_y,  text_str , size=10, rotation=15)        

    axs[5].set_xlabel(f'sec, (размер окна: {window_size}, {round(window_size/409,2)} sec) ')
    temp = []
    
    # получение базовых амплитуд
    rez_array_base = []
    
    # для V1 берется со 2 сегмента
    
    ch_offset = 0
    temp = record.channels[2].data_split[n_seg + 1]
    temp.calculate_window_spectrum(peaks_height = peaks_height, n = window_size)
    
    if plot_base:
        fig, axs = plt.subplots(6, 1, figsize=(16, 6*1.5))
        fig.subplots_adjust(wspace=0.07, hspace=0)
        axs[0].set_title('Базовые участки для каждого канала  (сборный, на каждый канал свой сегмент, V1 - второй, остальные - первый)', fontsize=12)
        
        axs[ch_offset].plot(np.arange(temp.data.size)/409, temp.data, color=plot_color)
        
        if temp.check_spike():
                axs[ch_offset].set_facecolor(color_spikes)
                check_spikes_warnung_flag = True
        
        axs[ch_offset].grid(True) 
        axs[ch_offset].set_ylabel(f'{record.channels[ch_offset+2].name}, мВ', rotation=0, fontsize=12)
        
    if temp.window_spectrum_array.size !=0:
        grap = temp.window_spectrum_array[:,0]
        rez_array_base.append(np.median(np.array(grap[-n_size:])))
        if plot_base:
            freq = temp.window_spectrum_array[:,1]
            if len(grap)//7 == 0:
                step = 1
            else:
                step = len(grap)//7
            axs[ch_offset].set_ylim(top = axs[ch_offset].get_ylim()[1]*1.3)
            for index in np.arange(0 ,len(grap), step):
                text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                limit_diff = axs[ch_offset].get_ylim()[1] - axs[ch_offset].get_ylim()[0]
                text_y = axs[ch_offset].get_ylim()[1] - limit_diff * 0.4                
                axs[ch_offset].text(index*temp.step/409 , text_y,  text_str , size=10, rotation=15)   
        
    
    # для V2-V6 каналов берется с 1 сегмента
    for ch_offset in range(1,6):
        temp = record.channels[ch_offset+2].data_split[n_seg]
        temp.calculate_window_spectrum(peaks_height = peaks_height, n = window_size)
        if plot_base:
            axs[ch_offset].plot(np.arange(temp.data.size)/409, temp.data, color=plot_color)
            axs[ch_offset].grid(True) 
            
            if temp.check_spike():
                axs[ch_offset].set_facecolor(color_spikes)
                check_spikes_warnung_flag = True
            
            axs[ch_offset].set_ylabel(f'{record.channels[ch_offset+2].name}, мВ', rotation=0, fontsize=12)
        if temp.window_spectrum_array.size !=0:
            grap = temp.window_spectrum_array[:,0]
            rez_array_base.append(np.median(np.array(grap[-n_size:])))
            if plot_base:
                freq = temp.window_spectrum_array[:,1]
                if len(grap)//7 == 0:
                    step = 1
                else:
                    step = len(grap)//7
                axs[ch_offset].set_ylim(top = axs[ch_offset].get_ylim()[1]*1.3)
                for index in np.arange(0 ,len(grap), step):
                    text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                    limit_diff = axs[ch_offset].get_ylim()[1] - axs[ch_offset].get_ylim()[0]
                    text_y = axs[ch_offset].get_ylim()[1] - limit_diff * 0.4                
                    axs[ch_offset].text(index*temp.step/409 , text_y,  text_str , size=10, rotation=15)   
                    
    axs[5].set_xlabel(f'sec, (размер окна: {window_size}, {round(window_size/409,2)} sec) ')        
    # расчет сопротивлений
    impedanse_array = []
    voltage_array = []
    for i_ch in range(6):
        voltage_array.append([round(rez_array[i_ch],3), round(rez_array_base[i_ch],3)])
        impedanse_array.append(round(0.62*rez_array[i_ch]/(rez_array_base[i_ch]-rez_array[i_ch])))
    
    
    print('Значения напряжения:',voltage_array)       
    print('Полученные входные сопротивления:',impedanse_array)
    plt.show()
    return impedanse_array, check_spikes_warnung_flag